# Lesson 5: Leveraging Assistants API for SQL Databases

## Import the helper function

To access the ``Helper.py`` file, please go to the ``File`` menu and select ``Open...``.

In [23]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


In [1]:
import Helper
from Helper import get_positive_cases_for_state_on_date
from Helper import get_hospitalized_increase_for_state_on_date

## Launch the Assistant API

**Note**: The pre-configured cloud resource grants you access to the Azure OpenAI GPT model. The key and endpoint provided below are intended for teaching purposes only. Your notebook environment is already set up with the necessary keys, which may differ from those used by the instructor during the filming.

In [2]:
from openai import AzureOpenAI
import json
import os

from dotenv import load_dotenv
load_dotenv()

azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT")
azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
api_key=os.getenv("AZURE_OPENAI_KEY")

client = AzureOpenAI(
    api_version="2024-05-01-preview",
    azure_endpoint=azure_endpoint,
    api_key=api_key
)

# I) Create assistant
assistant = client.beta.assistants.create(
  name="Covid Assistant",
  instructions="""You are an assistant answering questions 
                  about a Covid dataset.""",
  model=azure_deployment, 
  tools=Helper.tools_sql)

# II) Create thread
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_Lb61Nx93VYw9qZ2ovkX9an84', created_at=1729003929, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [3]:
# III) Add message
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="""how many hospitalized people we had in Alaska
               the 2021-03-05?"""
)
print(message)

Message(id='msg_kxG9o8XgDuL19mVEEDaTZAcl', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='how many hospitalized people we had in Alaska\n               the 2021-03-05?'), type='text')], created_at=1729003932, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_Lb61Nx93VYw9qZ2ovkX9an84')


In [4]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_kxG9o8XgDuL19mVEEDaTZAcl",
      "assistant_id": null,
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "how many hospitalized people we had in Alaska\n               the 2021-03-05?"
          },
          "type": "text"
        }
      ],
      "created_at": 1729003932,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "user",
      "run_id": null,
      "status": null,
      "thread_id": "thread_Lb61Nx93VYw9qZ2ovkX9an84"
    }
  ],
  "object": "list",
  "first_id": "msg_kxG9o8XgDuL19mVEEDaTZAcl",
  "last_id": "msg_kxG9o8XgDuL19mVEEDaTZAcl",
  "has_more": false
}


In [5]:
# IV) Run assistant on thread

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

## Leverage the function calling with Assistants API

In [6]:
import time
from IPython.display import clear_output

start_time = time.time()

status = run.status

while status not in ["completed", "cancelled", "expired", "failed"]:
    time.sleep(5)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    print("Elapsed time: {} minutes {} seconds".format(
        int((time.time() - start_time) // 60),
        int((time.time() - start_time) % 60))
         )
    status = run.status
    print(f'Status: {status}')
    if (status=="requires_action"):
        available_functions = {
            "get_positive_cases_for_state_on_date": get_positive_cases_for_state_on_date,
            "get_hospitalized_increase_for_state_on_date":get_hospitalized_increase_for_state_on_date
        }

        tool_outputs = []
        for tool_call in run.required_action.submit_tool_outputs.tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                state_abbr=function_args.get("state_abbr"),
                specific_date=function_args.get("specific_date"),
            )
            print(function_response)
            print(tool_call.id)
            tool_outputs.append(
                { "tool_call_id": tool_call.id,
                 "output": str(function_response)
                }
            )

        run = client.beta.threads.runs.submit_tool_outputs(
          thread_id=thread.id,
          run_id=run.id,
          tool_outputs = tool_outputs
        )


messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages)

Elapsed time: 0 minutes 5 seconds
Status: requires_action
{'date': '2021-03-05', 'hospitalizedIncrease': 3}
call_7smTa9jbe6RXu8z0GPJKmRTF
Elapsed time: 0 minutes 11 seconds
Status: completed
SyncCursorPage[Message](data=[Message(id='msg_jQpmySbbATb99Ym0hB73s4FW', assistant_id='asst_bxcy6miyo8igfJ4pt9PJBGDg', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='On March 5, 2021, there was an increase of 3 hospitalizations in Alaska.'), type='text')], created_at=1729003952, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_B2S1j0jSihokBqcaSYWwcHto', status=None, thread_id='thread_Lb61Nx93VYw9qZ2ovkX9an84'), Message(id='msg_kxG9o8XgDuL19mVEEDaTZAcl', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='how many hospitalized people we had in Alaska\n               the 2021-03-05?'), type='text')], created_at=1729003932, incomple

In [7]:
print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_jQpmySbbATb99Ym0hB73s4FW",
      "assistant_id": "asst_bxcy6miyo8igfJ4pt9PJBGDg",
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "On March 5, 2021, there was an increase of 3 hospitalizations in Alaska."
          },
          "type": "text"
        }
      ],
      "created_at": 1729003952,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_B2S1j0jSihokBqcaSYWwcHto",
      "status": null,
      "thread_id": "thread_Lb61Nx93VYw9qZ2ovkX9an84"
    },
    {
      "id": "msg_kxG9o8XgDuL19mVEEDaTZAcl",
      "assistant_id": null,
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "how many hospitalized people we had in Alaska\n       

## Add the code interpreter

In [61]:
file = client.files.create(
  file=open("./data/all-states-history.csv", "rb"),
  purpose='assistants'
)

assistant = client.beta.assistants.create(
  instructions="""You are an assistant answering questions about a Covid dataset.""",
  model="gpt-4", 
  tools=[{"type":"code_interpreter"}],
  tool_resources={"code_interpreter":{"file_ids":[file.id]}}
)

thread = client.beta.threads.create()
print(thread)
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="""how many hospitalized people we had in Alaska the 2021-03-05?"""
)
print(message)
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)



Thread(id='thread_KVyHhmg0FCT7LFsva7nm1WLr', created_at=1728964873, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))
Message(id='msg_PMXaWWI1eHl9ZRsE15goulhy', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='how many hospitalized people we had in Alaska\n               the 2021-03-05?'), type='text')], created_at=1728964873, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_KVyHhmg0FCT7LFsva7nm1WLr')


In [62]:
status = run.status
start_time = time.time()
while status not in ["completed", "cancelled", "expired", "failed"]:
    time.sleep(5)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    print("Elapsed time: {} minutes {} seconds".format(
        int((time.time() - start_time) // 60),
        int((time.time() - start_time) % 60))
         )
    status = run.status
    print(f'Status: {status}')
    clear_output(wait=True)


messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_AJY32HJSEY3pPp1wO8Yk2PgW",
      "assistant_id": "asst_E3DxUkIEtQsqggZmQ3t1E454",
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "On March 5th, 2021, there were 1,293 people hospitalized due to Covid in Alaska according to the dataset."
          },
          "type": "text"
        }
      ],
      "created_at": 1728964892,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_U0k20tNaMFVc5IBarzOwB0y0",
      "status": null,
      "thread_id": "thread_KVyHhmg0FCT7LFsva7nm1WLr"
    },
    {
      "id": "msg_tMfGGFl6mSoLWU1A5BoBXeMU",
      "assistant_id": "asst_E3DxUkIEtQsqggZmQ3t1E454",
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "val

# Code from Azure AI Studio

In [52]:
import os
import json
import requests
import time
from openai import AzureOpenAI

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
  api_key= os.getenv("AZURE_OPENAI_KEY"),
  api_version="2024-05-01-preview"
)

file = client.files.create(
  file=open("./data/all-states-history.csv", "rb"),
  purpose='assistants'
)

assistant = client.beta.assistants.create(
  model=azure_deployment, # replace with model deployment name.
  instructions="""You are an assistant answering questions about a Covid dataset.""",
  tools=[{"type":"code_interpreter"}],
  tool_resources={"code_interpreter":{"file_ids":[file.id]}},
  temperature=1,
  top_p=1
)


In [53]:
# Create a thread
thread = client.beta.threads.create()

# Add a user question to the thread
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="how many hospitalized people we had in Alaska the 2021-03-05?" # Replace this with your prompt
)



# Run the thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

# Looping until the run completes or fails
while run.status in ['queued', 'in_progress', 'cancelling']:
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
elif run.status == 'requires_action':
  # the assistant requires calling some functions
  # and submit the tool outputs back to the run
  pass
else:
  print(run.status)


SyncCursorPage[Message](data=[Message(id='msg_bYf1n5bvrU9Dbk4sf7QJzKa5', assistant_id='asst_MS2g82tBnTZH3ULro39oaYNc', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='On March 5, 2021, there were 1,293 individuals who had been hospitalized due to Covid-19 in Alaska.'), type='text')], created_at=1728964703, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_FnkNJtti7msj91ymhsSd7cxL', status=None, thread_id='thread_gjCyrCOo0ASoQr9c12LDxiK2'), Message(id='msg_NwXg0BWvm5jatK1sfEMp6ift', assistant_id='asst_MS2g82tBnTZH3ULro39oaYNc', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="The data set is well-structured and contains a lot of information about Covid-19 cases across states. Let's filter this data for Alaska (AK) and the date 2021-03-05. Then we can look at the 'hospitalized' field for that date."), type='text')], created_at=172896469

In [73]:
#print(messages.model_dump_json(indent=2))
print(messages.data[0].content)

[TextContentBlock(text=Text(annotations=[], value='On March 5th, 2021, there were 1,293 people hospitalized due to Covid in Alaska according to the dataset.'), type='text')]
